In [3]:
import pybamm
import time
import numpy as np
import jax
import jax.numpy as jnp

2024-09-03 16:08:47.483 - [WARNING] citations.read_citations(76): Citations could not be read because the 'pybtex' library is not installed. Install 'pybamm[cite]' to enable citation reading.


In [20]:
# 1) pip install "pybamm[iree,jax]"  
# 2) then install after "pip install jax[cuda12]" - this upgrades jax to support CUDA12
# CUDA supported jax solver
print("Available devices:", jax.devices())

Available devices: [cuda(id=0)]


In [28]:
# We will want to differentiate our model, so let's define two input parameters
inputs = {
    "Current function [A]": 0.222,
    "Separator porosity": 0.3,
    
}

# Set-up the model
model = pybamm.lithium_ion.DFN()
geometry = model.default_geometry
param = model.default_parameter_values
param.update({key: "[input]" for key in inputs.keys()})
param.process_geometry(geometry)
param.process_model(model)
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p: 20, var.r_n: 10, var.r_p: 10}
mesh = pybamm.Mesh(geometry, model.default_submesh_types, var_pts)
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Use a short time-vector for this example, and declare which variables to track
t_eval = np.linspace(0, 360, 10)
output_variables = [
    "Voltage [V]",
    "Current [A]",
    "Time [min]",
]

# Create the IDAKLU Solver object
idaklu_solver = pybamm.IDAKLUSolver(
    rtol=1e-6,
    atol=1e-6,
    output_variables=output_variables,
)

In [29]:
# This is how we would normally perform a solve using IDAKLU
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

# Instead, we Jaxify the IDAKLU solver using similar arguments...
jax_solver = idaklu_solver.jaxify(
    model,
    t_eval,
)

# ... and then obtain a JAX expression for the solve
f = jax_solver.get_jaxpr()
print(f"JAX expression: {f}")

JAX expression: <function IDAKLUJax._jaxify.<locals>.f at 0x7fb6daea37f0>


In [23]:
# This is how we would normally perform a solve using IDAKLU
sim = idaklu_solver.solve(
    model,
    t_eval,
    inputs=inputs,
    calculate_sensitivities=True,
)

# Instead, we Jaxify the IDAKLU solver using similar arguments...
jax_solver = idaklu_solver.jaxify(
    model,
    t_eval,
)

# ... and then obtain a JAX expression for the solve
f = jax_solver.get_jaxpr()
print(f"JAX expression: {f}")

JAX expression: <function IDAKLUJax._jaxify.<locals>.f at 0x7fb6d8d44d30>


In [30]:
# Print all output variables, evaluated over a given time vector
data = f(t_eval, inputs)
print(data)

[[3.81933939e+000 2.22000000e-001 1.15631341e-311]
 [3.81349276e+000 2.22000000e-001 6.66666667e-001]
 [3.81083261e+000 2.22000000e-001 1.33333333e+000]
 [3.80888692e+000 2.22000000e-001 2.00000000e+000]
 [3.80717743e+000 2.22000000e-001 2.66666667e+000]
 [3.80555556e+000 2.22000000e-001 3.33333333e+000]
 [3.80397075e+000 2.22000000e-001 4.00000000e+000]
 [3.80240520e+000 2.22000000e-001 4.66666667e+000]
 [3.80085126e+000 2.22000000e-001 5.33333333e+000]
 [3.79930646e+000 2.22000000e-001 6.00000000e+000]]


In [31]:
# Isolate a single variables
data = jax_solver.get_var("Voltage [V]")(t_eval, inputs)
print(f"Isolating a single variable returns an array of shape {data.shape}")
print(data)

# Isolate two variables from the solver
data = jax_solver.get_vars(
    [
        "Voltage [V]",
        "Current [A]",
    ],
)(t_eval, inputs)
print(f"\nIsolating two variables returns an array of shape {data.shape}")
print(data)

Isolating a single variable returns an array of shape (10,)
[3.81933939 3.81349276 3.81083261 3.80888692 3.80717743 3.80555556
 3.80397075 3.8024052  3.80085126 3.79930646]

Isolating two variables returns an array of shape (10, 2)
[[3.81933939 0.222     ]
 [3.81349276 0.222     ]
 [3.81083261 0.222     ]
 [3.80888692 0.222     ]
 [3.80717743 0.222     ]
 [3.80555556 0.222     ]
 [3.80397075 0.222     ]
 [3.8024052  0.222     ]
 [3.80085126 0.222     ]
 [3.79930646 0.222     ]]


In [26]:
# Calculate the Jacobian matrix (via forward autodiff)
t_start = time.time()
out = jax.jacfwd(f, argnums=1)(t_eval, inputs)
print(f"Jacobian forward method ran in {time.time()-t_start:0.3} secs")
print(out)

# Calculate Jacobian matrix (via backward autodiff)
t_start = time.time()
out = jax.jacrev(f, argnums=1)(t_eval, inputs)
print(f"\nJacobian reverse method ran in {time.time()-t_start:0.3} secs")
print(out)

Jacobian forward method ran in 0.0682 secs
{'Current function [A]': Array([[-0.1192102 ,  1.        ,  0.        ],
       [-0.13512293,  1.        ,  0.        ],
       [-0.14406751,  1.        ,  0.        ],
       [-0.15186641,  1.        ,  0.        ],
       [-0.15920157,  1.        ,  0.        ],
       [-0.16630362,  1.        ,  0.        ],
       [-0.17326745,  1.        ,  0.        ],
       [-0.18013099,  1.        ,  0.        ],
       [-0.18690734,  1.        ,  0.        ],
       [-0.1935984 ,  1.        ,  0.        ]], dtype=float64), 'Separator porosity': Array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float64)}

Jacobian reverse method ran in 1.05 secs
{'Current function [A]': Array([[-0.1192102 ,  1.        ,  0.        ],
       [-0.13512293,  1.        ,  0.        ],
       [-0.14406751,  1

In [32]:
# Isolate the derivate of Voltage with respect to the Current function:
out = jax.jacfwd(f, argnums=1)(t_eval, inputs)
data = jax_solver.get_var(out["Current function [A]"], "Voltage [V]")
print(data)

[-0.13629603 -0.16386375 -0.17615635 -0.18494974 -0.19258651 -0.19978548
 -0.20678279 -0.21365524 -0.22043194 -0.22711774]


In [18]:
# Example evaluation using the `grad` function
t_start = time.time()
data = jax.vmap(
    jax.grad(
        jax_solver.get_var("Voltage [V]"),
        argnums=1,  # take derivative with respect to `inputs`
    ),
    in_axes=(0, None),  # map time over the 0th dimension and do not map inputs
)(t_eval, inputs)
print(f"Gradient method ran in {time.time()-t_start:0.3} secs")
print(data)

'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring feature)
'+ptx86' is not a recognized feature for this target (ignoring f

Gradient method ran in 0.83 secs
{'Current function [A]': Array([-0.13629603, -0.16386375, -0.17615635, -0.18494974, -0.19258651,
       -0.19978548, -0.20678279, -0.21365524, -0.22043194, -0.22711774],      dtype=float64), 'Separator porosity': Array([0.00579553, 0.0079704 , 0.0095279 , 0.01024855, 0.01053721,
       0.01064576, 0.0106863 , 0.01070118, 0.01070757, 0.01071102],      dtype=float64)}


# use case example

As a use-case example, consider a fitting procedure where we want to compare simulation data against some experimental data. We achieve this by computing the sum-of-squared error (SEE) between the two. Many fitting procedures will converge more quickly (with fewer iterations) if both the value and gradient of the SSE function are provided. By making use of JAX-expressions we can derive these effortlessly.

Note: We do not need to map over time when calling value_and_grad in this example as the sse function returns a scalar (despite taking vector inputs).

In [33]:
# Simulate some experimental data using our original parameter settings
data = sim["Voltage [V]"](t_eval)


# Sum-of-squared errors
def sse(t, inputs):
    modelled = jax_solver.get_var("Voltage [V]")(t_eval, inputs)
    return jnp.sum((modelled - data) ** 2)


# Provide some predicted model inputs (these could come from a fitting procedure)
inputs_pred = {
    "Current function [A]": 0.150,
    "Separator porosity": 0.333,
}

# Get the value and gradient of the SSE function
t_start = time.time()
value, gradient = jax.value_and_grad(sse, argnums=1)(t_eval, inputs_pred)
print(f"Value and gradient computed in {time.time()-t_start:0.3} secs")
print("SSE value: ", value)
print("SSE gradient (wrt each input): ", gradient)

Value and gradient computed in 0.142 secs
SSE value:  0.00208164995228154
SSE gradient (wrt each input):  {'Current function [A]': array(-0.05767217), 'Separator porosity': array(0.0014688)}
